In [1]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
import re
import pandas as pd
from io import BytesIO
import unicodedata

!pip install rapidfuzz

from rapidfuzz import fuzz, process

In [2]:
base_url = "https://www.comunidad.madrid/servicios/educacion/publicaciones-interes-universitario"
html = requests.get(base_url).text
soup = BeautifulSoup(html, "html.parser")

pdf_links = [
    a["href"] for a in soup.find_all("a", href=True)
    if "notas" in a["href"].lower() and a["href"].endswith(".pdf")
]

if not pdf_links:
    raise Exception("No se encontró el PDF de notas de corte en la página.")

pdf_url = pdf_links[0]
if not pdf_url.startswith("http"):
    pdf_url = "https://www.comunidad.madrid" + pdf_url

response = requests.get(pdf_url)
pdf_bytes = BytesIO(response.content)

filas = []
año = 2025
convocatoria = "ordinaria"

with pdfplumber.open(pdf_bytes) as pdf:
    universidad_actual = None
    for page in pdf.pages:
        texto = page.extract_text()
        if not texto:
            continue
        uni_match = re.search(r"(UNIVERSIDAD\s+[A-ZÁÉÍÓÚÑ\s]+?)(?:CURSO|\n|$)", texto)
        if uni_match:
            universidad_actual = uni_match.group(1).title().strip()
        table = page.extract_table()
        if not table:
            continue
        headers = table[0]
        data_rows = table[1:]
        for row in data_rows:
            if not any(row):
                continue
            codigo, grado, *resto = row
            if not grado or grado.lower().startswith("código"):
                continue
            for i, nota in enumerate(resto):
                if nota and re.search(r"\d", nota):
                    grupo = f"Grupo {i//2 + 1}"
                    valor = re.search(r"[\d,]+", nota)
                    if valor:
                        filas.append({
                            "universidad": universidad_actual,
                            "grado": grado.strip(),
                            "grupo": grupo,
                            "nota": valor.group(0).replace(",", "."),
                            "año": año,
                            "convocatoria": convocatoria
                        })

df_notas = pd.DataFrame(filas)
df_notas.drop_duplicates(subset=["universidad", "grado", "grupo"], inplace=True)

def agrupar_similares(valores, umbral=85, umbral_siglas=85):
    valores_unicos = list(valores.dropna().unique())
    grupos = {}
    usados = set()
    def siglas(texto):
        return ''.join([c for c in texto if c.isupper()])
    for v in valores_unicos:
        if v in usados:
            continue
        grupo = [v]
        usados.add(v)
        siglas_v = siglas(v)
        for otro in valores_unicos:
            if otro in usados:
                continue
            simil = fuzz.token_sort_ratio(v, otro)
            siglas_otro = siglas(otro)
            if simil >= umbral:
                grupo.append(otro)
                usados.add(otro)
                continue
            if siglas_v and siglas_otro:
                simil_siglas = fuzz.ratio(siglas_v, siglas_otro)
                if simil_siglas >= umbral_siglas:
                    grupo.append(otro)
                    usados.add(otro)
                    continue
        cambio = grupo[0]
        for previo in grupo:
            grupos[previo] = cambio
    return grupos

def normalizar_texto_general(texto):
    if not isinstance(texto, str):
        return texto
    texto = texto.strip().lower()
    texto = ''.join(c for c in unicodedata.normalize('NFKD', texto) if not unicodedata.combining(c))
    texto = re.sub(r'\([^)]*\)', '', texto)
    texto = re.sub(r'\s*-\s*', ' - ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    texto = texto.title()
    excepciones = ["De", "Del", "La", "Las", "El", "Los", "Y", "En", "Por", "Para"]
    for exc in excepciones:
        texto = re.sub(rf'\b{exc}\b', exc.lower(), texto)
    return texto

def normalizar_universidad(texto):
    if not isinstance(texto, str):
        return texto
    texto = texto.strip()
    texto = re.sub(r'\s+', ' ', texto)
    texto = texto.title()
    return texto

mapa_universidades = agrupar_similares(df_notas['universidad'])
df_notas['universidad'] = df_notas['universidad'].map(mapa_universidades).fillna(df_notas['universidad'])
df_notas['universidad'] = df_notas['universidad'].apply(normalizar_universidad)
df_notas['grado'] = df_notas['grado'].apply(normalizar_texto_general)

df_notas_expandidas = []
for _, row in df_notas.iterrows():
    grados = [g.strip() for g in row["grado"].split(" - ")]
    for g in grados:
        nueva = row.copy()
        nueva["grado"] = g
        df_notas_expandidas.append(nueva)

df_notas = pd.DataFrame(df_notas_expandidas)
df_notas.to_csv("../Input/notas_corte.csv", index=False, encoding="utf-8")